# Description
This notebook will experiment with:
- train & compare differnt GNN models for graph classificaiton in common benchmarks (PPI, Proteins, ENZYMES,..)
- compare results to publication results

Most of the experiments will be done in PyTorch/PyTorch Geometric, but some models are implemented in Tensor Flow.

In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import MessagePassing
#from torch_geometric.nn.conv.gated_graph_conv import GatedGraphConv
from torch_geometric.nn.glob.glob import global_mean_pool, global_add_pool
import torch.nn as nn
from torch.nn import Sequential as Seq, Linear as Lin, ReLU
from torch_scatter import scatter_mean
from torch_geometric.nn import MetaLayer

from TFM_graph_classification import *

# 1. Models

In [2]:

class GGNN1(torch.nn.Module):
    def __init__(self, d1=50,d2=20,num_classes=6, num_layers=2, aggr_type='mean'):
        super(GGNN1, self).__init__()
        self.ggnn = GatedGraphConv(out_channels=d1, num_layers=num_layers,aggr=aggr_type, bias=True)
        self.fc1 = nn.Linear(d1, d2)
        self.fc2 = nn.Linear(d2, num_classes)
        self.global_pool = global_mean_pool
        
        

    def forward(self, data):
        x, edge_index, batch_vector = data.x, data.edge_index, data.batch

        x = self.ggnn(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training) # until here the output is for each node
        
        x = self.global_pool(x, batch_vector) # this makes the output to be graph level?
        #x = self.fc1(x)
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc2(x)
        #x = self.pool1(x, batch )
        x = F.log_softmax(x, dim=1)
        #x = torch.argmax(x, dim=1)  # we output softmax to use the nll_loss
        
        return x
    
class GGNN2(torch.nn.Module):
    def __init__(self, d1=50,d2=20,num_classes=6, num_layers=2, aggr_type='mean'):
        super(GGNN2, self).__init__()
        self.ggnn = GatedGraphConv(out_channels=d1, num_layers=num_layers,aggr=aggr_type, bias=True)
        self.fc1 = nn.Linear(d1, num_classes)
        self.global_pool = global_mean_pool
        
    def forward(self, data):
        x, edge_index, batch_vector = data.x, data.edge_index, data.batch

        x = self.ggnn(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training) # until here the output is for each node
        
        x = self.global_pool(x, batch_vector) # this makes the output to be graph level?
        
        x = self.fc1(x)
        x = F.log_softmax(x, dim=1)
        return x
    
    
class GGNN3(torch.nn.Module):
    def __init__(self, d1=50,d2=20, d3=10,num_classes=6, num_layers=2, aggr_type='mean'):
        super(GGNN3, self).__init__()
        self.ggnn = GatedGraphConv(out_channels=d1, num_layers=num_layers,aggr=aggr_type, bias=True)
        self.fc1 = nn.Linear(d1, d2)
        self.dense1_bn = nn.BatchNorm1d(d2)
        self.fc2 = nn.Linear(d2, d3)
        self.dense2_bn = nn.BatchNorm1d(d3)
        self.fc3 = nn.Linear(d3, num_classes)
        self.global_pool = global_mean_pool
        
    def forward(self, data):
        x, edge_index, batch_vector = data.x, data.edge_index, data.batch

        x = self.ggnn(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training) # until here the output is for each node
        
        x = self.global_pool(x, batch_vector) # this makes the output to be graph level?
        x = F.relu(self.dense1_bn(self.fc1(x)))
        x = F.relu(self.dense2_bn(self.fc2(x)))
        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)
        return x
    
class GGNN4(torch.nn.Module):
    def __init__(self, d1=50,d2=20,num_classes=6, num_layers=2, aggr_type='mean'):
        super(GGNN4, self).__init__()
        self.ggnn = GatedGraphConv(out_channels=d1, num_layers=num_layers,aggr=aggr_type, bias=True)
        self.fc1 = nn.Linear(d1, d2)
        self.dense1_bn = nn.BatchNorm1d(d2)
        self.fc2 = nn.Linear(d2, num_classes)
        self.global_pool = global_mean_pool
        
        
    def forward(self, data):
        x, edge_index, batch_vector = data.x, data.edge_index, data.batch

        x = self.ggnn(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training) # until here the output is for each node
        
        x = self.global_pool(x, batch_vector) # this makes the output to be graph level?
        #x = self.fc1(x)
        x = F.relu(self.dense1_bn(self.fc1(x)))
        #x = F.relu(self.fc2(x))
        x = self.fc2(x)
        #x = self.pool1(x, batch )
        x = F.log_softmax(x, dim=1)
        #x = torch.argmax(x, dim=1)  # we output softmax to use the nll_loss
        
        return x
    
class META1(torch.nn.Module):
    def __init__(self, d1=3, d2=50, d3=15, d4 =15,d5=10,num_classes=6):
        super(META1, self).__init__()

        self.edge_mlp = Seq(Lin(d1*3, d2), ReLU(), Lin(d2, d3))
        self.node_mlp = Seq(Lin(d1*6, d2), ReLU(), Lin(d2, d3))
        self.global_mlp = Seq(Lin(d3+1, d2), ReLU(), Lin(d2, d3))
        
        self.fc1 = nn.Linear(d4, d5)
        self.dense1_bn = nn.BatchNorm1d(d5)
        self.fc2 = nn.Linear(d5, num_classes)
        self.dense2_bn = nn.BatchNorm1d(num_classes)
        self.global_pool = global_mean_pool

        def edge_model(source, target, edge_attr, u):
            # source, target: [E, F_x], where E is the number of edges.
            # edge_attr: [E, F_e]
            # u: [B, F_u], where B is the number of graphs.
            #print("edge_model")
            #print(source.size())
            #print(target.size())
            #print(edge_attr.size())
            out = torch.cat([source, target, edge_attr], dim=1)
            return self.edge_mlp(out)

        def node_model(x, edge_index, edge_attr, u):
            # x: [N, F_x], where N is the number of nodes.
            # edge_index: [2, E] with max entry N - 1.
            # edge_attr: [E, F_e]
            # u: [B, F_u]
            row, col = edge_index
            
            #print("node_model")
            #print(row.size())
            #print(col.size())
            #print(x[col].size())
            #print(edge_attr.size())
            
            out = torch.cat([x[col], edge_attr], dim=1)
            out = self.node_mlp(out)
            return scatter_mean(out, row, dim=0, dim_size=x.size(0))

        def global_model(x, edge_index, edge_attr, u, batch):
            # x: [N, F_x], where N is the number of nodes.
            # edge_index: [2, E] with max entry N - 1.
            # edge_attr: [E, F_e]
            # u: [B, F_u]
            # batch: [N] with max entry B - 1.
            
            #print("global_Model")
            #print("u.size():")
            #print(u.size())
            #print("scatter_mean(x,batch,..):")
            #smean = scatter_mean(x, batch, dim=0)
            #print(smean.size())
            
            out = torch.cat([u, scatter_mean(x, batch, dim=0)], dim=1)
            
            #print("out.size():")
            #print(out.size())
            return self.global_mlp(out)

        self.op = MetaLayer(edge_model, node_model, global_model)

    def forward(self, data):
        
        x, edge_index, edge_attr, u, batch = data.x, data.edge_index, data.edge_attr, data.u, data.batch        
        
        # output of meta is x,edge_attr, u
        x2, edge_attr2, u2 =  self.op(x, edge_index, edge_attr, u, batch)
        
        # idea1 is to cat x2, edge_attr2 and u2?
        # idea2 is to update edge_attr and u...
        data.x = x2
        data.edge_attr = edge_attr2
        data.u = u2

        # version using only u
        x = F.relu(self.dense1_bn(self.fc1(u2)))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x
        
    
    
class META2(torch.nn.Module):
    def __init__(self, d1=3, d2=50, d3=15, d4 =15,d5=10,num_classes=6):
        super(META2, self).__init__()

        self.edge_mlp = Seq(Lin(d1*3, d2), ReLU(), Lin(d2, d3))
        self.node_mlp = Seq(Lin(d1*6, d2), ReLU(), Lin(d2, d3))
        self.global_mlp = Seq(Lin(d3+1, d2), ReLU(), Lin(d2, d3))
        
        self.fc1 = nn.Linear(d4, d5)
        self.dense1_bn = nn.BatchNorm1d(d5)
        self.fc2 = nn.Linear(d5, num_classes)
        self.dense2_bn = nn.BatchNorm1d(num_classes)
        self.global_pool = global_mean_pool

        def edge_model(source, target, edge_attr, u):
            # source, target: [E, F_x], where E is the number of edges.
            # edge_attr: [E, F_e]
            # u: [B, F_u], where B is the number of graphs.
            #print("edge_model")
            #print(source.size())
            #print(target.size())
            #print(edge_attr.size())
            out = torch.cat([source, target, edge_attr], dim=1)
            return self.edge_mlp(out)

        def node_model(x, edge_index, edge_attr, u):
            # x: [N, F_x], where N is the number of nodes.
            # edge_index: [2, E] with max entry N - 1.
            # edge_attr: [E, F_e]
            # u: [B, F_u]

            
            row, col = edge_index
            
            #print("node_model")
            #print(row.size())
            #print(col.size())
            #print(x[col].size())
            #print(edge_attr.size())
            
            out = torch.cat([x[col], edge_attr], dim=1)
            out = self.node_mlp(out)
            return scatter_mean(out, row, dim=0, dim_size=x.size(0))

        def global_model(x, edge_index, edge_attr, u, batch):
            # x: [N, F_x], where N is the number of nodes.
            # edge_index: [2, E] with max entry N - 1.
            # edge_attr: [E, F_e]
            # u: [B, F_u]
            # batch: [N] with max entry B - 1.
            
            #print("global_Model")
            #print("u.size():")
            #print(u.size())
            #print("scatter_mean(x,batch,..):")
            #smean = scatter_mean(x, batch, dim=0)
            #print(smean.size())
            
            out = torch.cat([u, scatter_mean(x, batch, dim=0)], dim=1)
            
            #print("out.size():")
            #print(out.size())
            return self.global_mlp(out)

        self.op = MetaLayer(edge_model, node_model, global_model)

    def forward(self, data):
        
        x, edge_index, edge_attr, u, batch = data.x, data.edge_index, data.edge_attr, data.u, data.batch        
        
        # output of meta is x,edge_attr, u
        x2, edge_attr2, u2 =  self.op(x, edge_index, edge_attr, u, batch)
        
        # idea1 is to cat x2, edge_attr2 and u2?
        # idea2 is to update edge_attr and u...
        data.x = x2
        data.edge_attr = edge_attr2
        data.u = u2
        
        # version using only x 
        x = self.global_pool(x2,batch) # separate by graph level
        x = F.relu(self.dense1_bn(self.fc1(x)))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x
    

    
class META3(torch.nn.Module):
    def __init__(self, d1=3, d2=50, d3=15, d4 =15,d5=10,num_classes=6):
        super(META3, self).__init__()

        self.edge_mlp = Seq(Lin(d1*3, d2), ReLU(), Lin(d2, d3))
        self.node_mlp = Seq(Lin(d1*6, d2), ReLU(), Lin(d2, d3))
        self.global_mlp = Seq(Lin(d3+1, d2), ReLU(), Lin(d2, d3))
        
        self.fc1 = nn.Linear(d4, d5)
        self.dense1_bn = nn.BatchNorm1d(d5)
        self.fc2 = nn.Linear(d5, num_classes)
        self.dense2_bn = nn.BatchNorm1d(num_classes)
        self.global_pool = global_mean_pool

        def edge_model(source, target, edge_attr, u):
            # source, target: [E, F_x], where E is the number of edges.
            # edge_attr: [E, F_e]
            # u: [B, F_u], where B is the number of graphs.
            #print("edge_model")
            #print(source.size())
            #print(target.size())
            #print(edge_attr.size())
            out = torch.cat([source, target, edge_attr], dim=1)
            return self.edge_mlp(out)

        def node_model(x, edge_index, edge_attr, u):
            # x: [N, F_x], where N is the number of nodes.
            # edge_index: [2, E] with max entry N - 1.
            # edge_attr: [E, F_e]
            # u: [B, F_u]

            
            row, col = edge_index
            
            #print("node_model")
            #print(row.size())
            #print(col.size())
            #print(x[col].size())
            #print(edge_attr.size())
            
            out = torch.cat([x[col], edge_attr], dim=1)
            out = self.node_mlp(out)
            return scatter_mean(out, row, dim=0, dim_size=x.size(0))

        def global_model(x, edge_index, edge_attr, u, batch):
            # x: [N, F_x], where N is the number of nodes.
            # edge_index: [2, E] with max entry N - 1.
            # edge_attr: [E, F_e]
            # u: [B, F_u]
            # batch: [N] with max entry B - 1.
            
            #print("global_Model")
            #print("u.size():")
            #print(u.size())
            #print("scatter_mean(x,batch,..):")
            #smean = scatter_mean(x, batch, dim=0)
            #print(smean.size())
            
            out = torch.cat([u, scatter_mean(x, batch, dim=0)], dim=1)
            
            #print("out.size():")
            #print(out.size())
            return self.global_mlp(out)

        self.op = MetaLayer(edge_model, node_model, global_model)

    def forward(self, data):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        x, edge_index, edge_attr, u, batch = data.x, data.edge_index, data.edge_attr, data.u, data.batch        
        
        # output of meta is x,edge_attr, u
        x2, edge_attr2, u2 =  self.op(x, edge_index, edge_attr, u, batch)
        
        # idea1 is to cat x2, edge_attr2 and u2?
        # idea2 is to update edge_attr and u...
        data.x = x2
        data.edge_attr = edge_attr2
        data.u = u2


        # version using x and  u
        x = F.relu(torch.cat([x2,u2], dim=0))
        x = F.dropout(x, training=self.training) # until here the output is for each node
        ubatch = list(set([ elem.item() for elem in batch]))
        #print(ubatch)
        x = self.global_pool(x, torch.cat([batch,torch.LongTensor( ubatch).to(device) ],dim=0)) # this makes the output to be graph level?
        x = F.relu(self.dense1_bn(self.fc1(x)))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x
    
class META4(torch.nn.Module):
    def __init__(self, d1=3, d2=50, d3=15, d4 =15,d5=10,num_classes=6):
        super(META4, self).__init__()

        self.edge_mlp = Seq(Lin(d1*3, d2), ReLU(), Lin(d2, d3))
        self.node_mlp = Seq(Lin(d1*6, d2), ReLU(), Lin(d2, d3))
        self.global_mlp = Seq(Lin(16, d2), ReLU(), Lin(d2, d3))
        
        self.fc1 = nn.Linear(d4, d5)
        self.dense1_bn = nn.BatchNorm1d(d5)
        self.fc2 = nn.Linear(d5, num_classes)
        self.dense2_bn = nn.BatchNorm1d(num_classes)
        self.global_pool = global_mean_pool

        def edge_model(source, target, edge_attr, u):
            # source, target: [E, F_x], where E is the number of edges.
            # edge_attr: [E, F_e]
            # u: [B, F_u], where B is the number of graphs.
            #print("edge_model")
            #print(source.size())
            #print(target.size())
            #print(edge_attr.size())
            out = torch.cat([source, target, edge_attr], dim=1)
            return self.edge_mlp(out)

        def node_model(x, edge_index, edge_attr, u):
            # x: [N, F_x], where N is the number of nodes.
            # edge_index: [2, E] with max entry N - 1.
            # edge_attr: [E, F_e]
            # u: [B, F_u]

            
            row, col = edge_index
            
            #print("node_model")
            #print(row.size())
            #print(col.size())
            #print(x[col].size())
            #print(edge_attr.size())
            
            out = torch.cat([x[col], edge_attr], dim=1)
            out = self.node_mlp(out)
            return scatter_mean(out, row, dim=0, dim_size=x.size(0))

        def global_model(x, edge_index, edge_attr, u, batch):
            # x: [N, F_x], where N is the number of nodes.
            # edge_index: [2, E] with max entry N - 1.
            # edge_attr: [E, F_e]
            # u: [B, F_u]
            # batch: [N] with max entry B - 1.
            
            #print("global_Model")
            #print("u.size():")
            #print(u.size())
            #print("scatter_mean(x,batch,..):")
            #smean = scatter_mean(x, batch, dim=0)
            #print(smean.size())
            
            out = torch.cat([u, scatter_mean(x, batch, dim=0)], dim=1)
            
            #print("out.size():")
            #print(out.size())
            return self.global_mlp(out)

        self.op = MetaLayer(edge_model, node_model, global_model)

    def forward(self, data):
        
        x, edge_index, edge_attr, u, batch = data.x, data.edge_index, data.edge_attr, data.u, data.batch        
        
        # output of meta is x,edge_attr, u
        x2, edge_attr2, u2 =  self.op(x, edge_index, edge_attr, u, batch)
        
        # idea1 is to cat x2, edge_attr2 and u2?
        # idea2 is to update edge_attr and u...
        data.x = x2
        data.edge_attr = edge_attr2
        data.u = u2

    
        # version using x and  u and edge_attr
        x = F.relu(torch.cat([x2,u2], dim=0))
        #x = x2
        x = F.dropout(x, training=self.training) # until here the output is for each node
        
        x = self.global_pool(x, torch.cat([batch, ],dim=0)) # this makes the output to be graph level?
        x = F.relu(self.dense1_bn(self.fc1(x)))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x
    
    
class META5(torch.nn.Module):
    """
        Not using edge attribute
    """
    def __init__(self, d1=3, d2=50, d3=15):
        super(META5, self).__init__()

        self.edge_mlp = Seq(Lin(d1*2, d2), ReLU(), Lin(d2, d3))
        self.node_mlp = Seq(Lin(d1, d2), ReLU(), Lin(d2, d3))
        self.global_mlp = Seq(Lin(2, d2), ReLU(), Lin(d2, d3))

        def edge_model(source, target, edge_attr, u):
            # source, target: [E, F_x], where E is the number of edges.
            # edge_attr: [E, F_e]
            # u: [B, F_u], where B is the number of graphs.
            out = torch.cat([source, target], dim=1)
            #print("edge_model")
            #print(out.size())
            return self.edge_mlp(out)

        def node_model(x, edge_index, edge_attr, u):
            # x: [N, F_x], where N is the number of nodes.
            # edge_index: [2, E] with max entry N - 1.
            # edge_attr: [E, F_e]
            # u: [B, F_u]
            row, col = edge_index
            out = torch.cat([x[col]], dim=1)
            out = self.node_mlp(out)
            return scatter_mean(out, row, dim=0, dim_size=x.size(0))

        def global_model(x, edge_index, edge_attr, u, batch):
            # x: [N, F_x], where N is the number of nodes.
            # edge_index: [2, E] with max entry N - 1.
            # edge_attr: [E, F_e]
            # u: [B, F_u]
            # batch: [N] with max entry B - 1.
            out = torch.cat([u, scatter_mean(x, batch, dim=0)], dim=1)
            
            return self.global_mlp(out)

        self.op = MetaLayer(edge_model, node_model, global_model)

    def forward(self, x, edge_index, edge_attr, u, batch):
        #print("Forward: ")
        #print(x.size())
        return self.op(x, edge_index, edge_attr, u, batch)

In [3]:

model_list21 = []
for modelclass in [GGNN1]:
    for d1 in [25,50,100]:
        for d2 in [20,50]:
            for aggr_type in ['mean','add']:
                for epochs in [100,300,600]:
                    for num_layers in [2,4,8,10]:
                        for lr in [0.01]:
                            for wd in [5e-4]:
                                model_list21.append(
                                    {
                                    'model': modelclass,
                                    'epochs': epochs,
                                    'kwargs':{'d1': d1,'d2': d2,'num_layers':num_layers, 
                                              'aggr_type':aggr_type}, 
                                    'learning_rate': lr, 'weight_decay':wd, 
                                    'batch_size': 32},
                                )
model_list22 = []
for modelclass in [GGNN2]:
    for d1 in [25,50,100]:
        for d2 in [20,50]:
            for aggr_type in ['mean','add']:
                for epochs in [100,300,600]:
                    for num_layers in [2,4,8,10]:
                        for lr in [0.01]:
                            for wd in [5e-4]:
                                model_list22.append(
                                    {
                                    'model': modelclass,
                                    'epochs': epochs,
                                    'kwargs':{'d1': d1,'d2': d2,'num_layers':num_layers, 
                                              'aggr_type':aggr_type}, 
                                    'learning_rate': lr, 'weight_decay':wd, 
                                    'batch_size': 32},
                                )
                                
model_list23 = []
for modelclass in [GGNN3]:
    for d1 in [25,50,100]:
        for d2 in [20,50]:
            for aggr_type in ['mean','add']:
                for epochs in [100,300,600]:
                    for num_layers in [2,4,8,10]:
                        for lr in [0.01]:
                            for wd in [5e-4]:
                                model_list23.append(
                                    {
                                    'model': modelclass,
                                    'epochs': epochs,
                                    'kwargs':{'d1': d1,'d2': d2,'num_layers':num_layers, 
                                              'aggr_type':aggr_type}, 
                                    'learning_rate': lr, 'weight_decay':wd, 
                                    'batch_size': 32},
                                )
                                
model_list24 = []
for modelclass in [GGNN4]:
    for d1 in [25,50,100]:
        for d2 in [20,50]:
            for aggr_type in ['mean','add']:
                for epochs in [100,300,600]:
                    for num_layers in [2,4,8,10]:
                        for lr in [0.01]:
                            for wd in [5e-4]:
                                model_list24.append(
                                    {
                                    'model': modelclass,
                                    'epochs': epochs,
                                    'kwargs':{'d1': d1,'d2': d2,'num_layers':num_layers, 
                                              'aggr_type':aggr_type}, 
                                    'learning_rate': lr, 'weight_decay':wd, 
                                    'batch_size': 32},
                                )
                                
model_list25 = []
for modelclass in [META3,]:
    for d1 in [3]:
        for d2 in [10,20,50,100]:
            for d3 in [15]:
                for d4 in [15]:
                    for d5 in [10,20]:
                        for epochs in [100,300,600]:
                            model_list25.append(
                                {'epochs': epochs,
                                'model': modelclass,
                                'kwargs':{'d1': d1,'d2': d2, 'd3': d3, 'd4': d4, 'd5':d5}, 
                                'learning_rate': 0.01, 'weight_decay':5e-4, 'batch_size': 64},
                            )
                                
model_list26 = []
for modelclass in [META2,]:
    for d1 in [3]:
        for d2 in [10,20,50,100]:
            for d3 in [15]:
                for d4 in [15]:
                    for d5 in [10,20]:
                        for epochs in [100,300,600]:
                            model_list26.append(
                                {'epochs': epochs,
                                'model': modelclass,
                                'kwargs':{'d1': d1,'d2': d2, 'd3': d3, 'd4': d4, 'd5':d5}, 
                                'learning_rate': 0.01, 'weight_decay':5e-4, 'batch_size': 64},
                            )
                              
                               



In [4]:

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
dataset = dataset.shuffle()
k = 3
n = len(dataset)
print(" n:",n," k folds=",k)
train_dataset, test_dataset = balancedDatasetSplit_slice(dataset, prop=0.8)
print("Datasets balancing: ")
printDatasetBalance(dataset )
printDatasetBalance(train_dataset )
printDatasetBalance(test_dataset )
print()

modelsdict = modelSelection(model_list21,k, train_dataset)
reportModelSelectionResult(modelsdict)
#!cp -r models /content/drive/My\ Drive/TFM/graph_classification/
#!cp -r results /content/drive/My\ Drive/TFM/graph_classification/


modelsdict2 = modelSelection(model_list22,k, train_dataset)
reportModelSelectionResult(modelsdict2)
#!cp -r models /content/drive/My\ Drive/TFM/graph_classification/
#!cp -r results /content/drive/My\ Drive/TFM/graph_classification/


modelsdict3 = modelSelection(model_list23,k, train_dataset)
reportModelSelectionResult(modelsdict3)
#!cp -r models /content/drive/My\ Drive/TFM/graph_classification/
#!cp -r results /content/drive/My\ Drive/TFM/graph_classification/


modelsdict4 = modelSelection(model_list24,k, train_dataset)
reportModelSelectionResult(modelsdict4)
#!cp -r models /content/drive/My\ Drive/TFM/graph_classification/
#!cp -r results /content/drive/My\ Drive/TFM/graph_classification/


modelsdict5 = modelSelection(model_list25,k, train_dataset)
reportModelSelectionResult(modelsdict5)
#!cp -r models /content/drive/My\ Drive/TFM/graph_classification/
#!cp -r results /content/drive/My\ Drive/TFM/graph_classification/


modelsdict6 = modelSelection(model_list26,k, train_dataset)
reportModelSelectionResult(modelsdict6)
#!cp -r models /content/drive/My\ Drive/TFM/graph_classification/
#!cp -r results /content/drive/My\ Drive/TFM/graph_classification/


modelsdict.update(modelsdict2)
modelsdict.update(modelsdict3)
modelsdict.update(modelsdict4)
modelsdict.update(modelsdict5)
modelsdict.update(modelsdict6)

bmodel = final_model_train(modelsdict['best_models']['loss'], train_dataset)
testresult = testModel(bmodel, test_dataset)
modelsdict['testing'][bmodel.__class__.__name__+'loss']=testresult

bmodel = final_model_train(modelsdict['best_models']['accuracy'], train_dataset)
testresult = testModel(bmodel, test_dataset)
modelsdict['testing'][bmodel.__class__.__name__+'accuracy']=testresult

bmodel = final_model_train(modelsdict['best_models']['microF1'], train_dataset)
testresult = testModel(bmodel, test_dataset)
modelsdict['testing'][bmodel.__class__.__name__+'microF1']=testresult

bmodel = final_model_train(modelsdict['best_models']['macroF1'], train_dataset)
testresult = testModel(bmodel, test_dataset)
modelsdict['testing'][bmodel.__class__.__name__+'macroF1']=testresult

reportAllTest(modelsdict)
saveResults(modelsdict)
#!cp -r models /content/drive/My\ Drive/TFM/graph_classification/
#!cp -r results /content/drive/My\ Drive/TFM/graph_classification/

 n: 600  k folds= 3
Datasets balancing: 
{0: 100, 1: 100, 2: 100, 3: 100, 4: 100, 5: 100}
{0: 80, 1: 80, 2: 80, 3: 80, 4: 80, 5: 80}
{0: 20, 1: 20, 2: 20, 3: 20, 4: 20, 5: 20}

 trained model:  GGNN1 {'d1': 25, 'd2': 20, 'num_layers': 2, 'aggr_type': 'mean'}  epochs: 100  val loss= 0.053266442070404686  val accuracy= 0.42857142857142855  val microF1= 0.38888888888888884  val macroF1= 0.3797539811107498
 trained model:  GGNN1 {'d1': 25, 'd2': 20, 'num_layers': 4, 'aggr_type': 'mean'}  epochs: 100  val loss= 0.05408225332697233  val accuracy= 0.2619047619047619  val microF1= 0.28846153846153844  val macroF1= 0.2533138182569345
 trained model:  GGNN1 {'d1': 25, 'd2': 20, 'num_layers': 8, 'aggr_type': 'mean'}  epochs: 100  val loss= 0.05742873251438141  val accuracy= 0.17857142857142858  val microF1= 0.16666666666666666  val macroF1= 0.04761904761904762
 trained model:  GGNN1 {'d1': 25, 'd2': 20, 'num_layers': 10, 'aggr_type': 'mean'}  epochs: 100  val loss= 0.05621028567353884  val accura

 trained model:  GGNN1 {'d1': 25, 'd2': 50, 'num_layers': 2, 'aggr_type': 'add'}  epochs: 100  val loss= 0.05273939669132233  val accuracy= 0.369047619047619  val microF1= 0.38461538461538464  val macroF1= 0.37893629869628215
 trained model:  GGNN1 {'d1': 25, 'd2': 50, 'num_layers': 4, 'aggr_type': 'add'}  epochs: 100  val loss= 0.05456084758043289  val accuracy= 0.3214285714285714  val microF1= 0.3461538461538461  val macroF1= 0.34642145653137807
 trained model:  GGNN1 {'d1': 25, 'd2': 50, 'num_layers': 8, 'aggr_type': 'add'}  epochs: 100  val loss= 0.05723257487018903  val accuracy= 0.2261904761904762  val microF1= 0.17521367521367523  val macroF1= 0.10316302885324581
 trained model:  GGNN1 {'d1': 25, 'd2': 50, 'num_layers': 10, 'aggr_type': 'add'}  epochs: 100  val loss= 0.05743002643187841  val accuracy= 0.14285714285714288  val microF1= 0.16666666666666666  val macroF1= 0.04761904761904762
 trained model:  GGNN1 {'d1': 25, 'd2': 50, 'num_layers': 2, 'aggr_type': 'add'}  epochs: 30

 trained model:  GGNN1 {'d1': 50, 'd2': 50, 'num_layers': 4, 'aggr_type': 'mean'}  epochs: 100  val loss= 0.054714409013589226  val accuracy= 0.2738095238095238  val microF1= 0.2884615384615385  val macroF1= 0.2743277932507049
 trained model:  GGNN1 {'d1': 50, 'd2': 50, 'num_layers': 8, 'aggr_type': 'mean'}  epochs: 100  val loss= 0.05744470531741778  val accuracy= 0.14285714285714282  val microF1= 0.1688034188034188  val macroF1= 0.06388842034095019
 trained model:  GGNN1 {'d1': 50, 'd2': 50, 'num_layers': 10, 'aggr_type': 'mean'}  epochs: 100  val loss= 0.05742922052741051  val accuracy= 0.11904761904761903  val microF1= 0.16666666666666666  val macroF1= 0.04761904761904762
 trained model:  GGNN1 {'d1': 50, 'd2': 50, 'num_layers': 2, 'aggr_type': 'mean'}  epochs: 300  val loss= 0.06313793361186981  val accuracy= 0.4880952380952381  val microF1= 0.4465811965811966  val macroF1= 0.4539758162159986
 trained model:  GGNN1 {'d1': 50, 'd2': 50, 'num_layers': 4, 'aggr_type': 'mean'}  epochs

 trained model:  GGNN1 {'d1': 100, 'd2': 20, 'num_layers': 4, 'aggr_type': 'add'}  epochs: 100  val loss= 0.05742888152599335  val accuracy= 0.14285714285714285  val microF1= 0.16666666666666666  val macroF1= 0.04779541446208113
 trained model:  GGNN1 {'d1': 100, 'd2': 20, 'num_layers': 8, 'aggr_type': 'add'}  epochs: 100  val loss= 0.05742928013205528  val accuracy= 0.17857142857142858  val microF1= 0.16666666666666666  val macroF1= 0.04761904761904762
 trained model:  GGNN1 {'d1': 100, 'd2': 20, 'num_layers': 10, 'aggr_type': 'add'}  epochs: 100  val loss= 0.05742854128281275  val accuracy= 0.25  val microF1= 0.16666666666666666  val macroF1= 0.04761904761904762
 trained model:  GGNN1 {'d1': 100, 'd2': 20, 'num_layers': 2, 'aggr_type': 'add'}  epochs: 300  val loss= 0.06099105874697367  val accuracy= 0.39285714285714285  val microF1= 0.41452991452991456  val macroF1= 0.41585144534700086
 trained model:  GGNN1 {'d1': 100, 'd2': 20, 'num_layers': 4, 'aggr_type': 'add'}  epochs: 300  va

 trained model:  GGNN2 {'d1': 25, 'd2': 20, 'num_layers': 2, 'aggr_type': 'mean'}  epochs: 100  val loss= 0.053238200644652046  val accuracy= 0.5238095238095238  val microF1= 0.39316239316239315  val macroF1= 0.39191720997878693
 trained model:  GGNN2 {'d1': 25, 'd2': 20, 'num_layers': 4, 'aggr_type': 'mean'}  epochs: 100  val loss= 0.05444942663113276  val accuracy= 0.30952380952380953  val microF1= 0.2884615384615385  val macroF1= 0.2844436386547075
 trained model:  GGNN2 {'d1': 25, 'd2': 20, 'num_layers': 8, 'aggr_type': 'mean'}  epochs: 100  val loss= 0.05531049892306328  val accuracy= 0.20238095238095236  val microF1= 0.2670940170940171  val macroF1= 0.26308821110982167
 trained model:  GGNN2 {'d1': 25, 'd2': 20, 'num_layers': 10, 'aggr_type': 'mean'}  epochs: 100  val loss= 0.05723012238740921  val accuracy= 0.17857142857142858  val microF1= 0.18376068376068377  val macroF1= 0.15445639379079498
 trained model:  GGNN2 {'d1': 25, 'd2': 20, 'num_layers': 2, 'aggr_type': 'mean'}  epo

...

 trained model:  GGNN4 {'d1': 50, 'd2': 50, 'num_layers': 4, 'aggr_type': 'mean'}  epochs: 100  val loss= 0.0559717503686746  val accuracy= 0.25  val microF1= 0.3311965811965812  val macroF1= 0.32458869764560544
 trained model:  GGNN4 {'d1': 50, 'd2': 50, 'num_layers': 8, 'aggr_type': 'mean'}  epochs: 100  val loss= 0.05782415593663851  val accuracy= 0.15476190476190477  val microF1= 0.16666666666666666  val macroF1= 0.04761904761904762
 trained model:  GGNN4 {'d1': 50, 'd2': 50, 'num_layers': 10, 'aggr_type': 'mean'}  epochs: 100  val loss= 0.057733748108148575  val accuracy= 0.11904761904761905  val microF1= 0.16452991452991453  val macroF1= 0.04735203957402173
 trained model:  GGNN4 {'d1': 50, 'd2': 50, 'num_layers': 2, 'aggr_type': 'mean'}  epochs: 300  val loss= 0.05916274959842364  val accuracy= 0.46428571428571425  val microF1= 0.43162393162393164  val macroF1= 0.4294007095984021
 trained model:  GGNN4 {'d1': 50, 'd2': 50, 'num_layers': 4, 'aggr_type': 'mean'}  epochs: 300  val 

 trained model:  GGNN4 {'d1': 100, 'd2': 20, 'num_layers': 8, 'aggr_type': 'add'}  epochs: 100  val loss= 0.06443663189808528  val accuracy= 0.14285714285714285  val microF1= 0.1730769230769231  val macroF1= 0.06493538631183153
 trained model:  GGNN4 {'d1': 100, 'd2': 20, 'num_layers': 10, 'aggr_type': 'add'}  epochs: 100  val loss= 0.05774501586953799  val accuracy= 0.15476190476190477  val microF1= 0.1730769230769231  val macroF1= 0.09583406955160234
 trained model:  GGNN4 {'d1': 100, 'd2': 20, 'num_layers': 2, 'aggr_type': 'add'}  epochs: 300  val loss= 0.05343504870931307  val accuracy= 0.4285714285714286  val microF1= 0.36324786324786323  val macroF1= 0.3785053481656297
 trained model:  GGNN4 {'d1': 100, 'd2': 20, 'num_layers': 4, 'aggr_type': 'add'}  epochs: 300  val loss= 0.05363605419794718  val accuracy= 0.3333333333333333  val microF1= 0.297008547008547  val macroF1= 0.2697574502193512
 trained model:  GGNN4 {'d1': 100, 'd2': 20, 'num_layers': 8, 'aggr_type': 'add'}  epochs: 

 trained model:  META3 {'d1': 3, 'd2': 10, 'd3': 15, 'd4': 15, 'd5': 10}  epochs: 100  val loss= 0.03148702532052994  val accuracy= 0.39285714285714285  val microF1= 0.33547008547008544  val macroF1= 0.33178173212578804
 trained model:  META3 {'d1': 3, 'd2': 10, 'd3': 15, 'd4': 15, 'd5': 10}  epochs: 300  val loss= 0.031116655717293423  val accuracy= 0.3333333333333333  val microF1= 0.3547008547008547  val macroF1= 0.35462382182552715
 trained model:  META3 {'d1': 3, 'd2': 10, 'd3': 15, 'd4': 15, 'd5': 10}  epochs: 600  val loss= 0.03156632992128531  val accuracy= 0.35714285714285715  val microF1= 0.3803418803418803  val macroF1= 0.37805977127401896
 trained model:  META3 {'d1': 3, 'd2': 10, 'd3': 15, 'd4': 15, 'd5': 20}  epochs: 100  val loss= 0.03107166290283203  val accuracy= 0.38095238095238093  val microF1= 0.3803418803418803  val macroF1= 0.3754061840041587
 trained model:  META3 {'d1': 3, 'd2': 10, 'd3': 15, 'd4': 15, 'd5': 20}  epochs: 300  val loss= 0.03150520039101442  val ac

 trained model:  META2 {'d1': 3, 'd2': 20, 'd3': 15, 'd4': 15, 'd5': 20}  epochs: 600  val loss= 0.04701860745747884  val accuracy= 0.2976190476190476  val microF1= 0.282051282051282  val macroF1= 0.2869390991168898
 trained model:  META2 {'d1': 3, 'd2': 50, 'd3': 15, 'd4': 15, 'd5': 10}  epochs: 100  val loss= 0.04195128381252289  val accuracy= 0.2023809523809524  val microF1= 0.235042735042735  val macroF1= 0.22849203347191635
 trained model:  META2 {'d1': 3, 'd2': 50, 'd3': 15, 'd4': 15, 'd5': 10}  epochs: 300  val loss= 0.04608990500370661  val accuracy= 0.30952380952380953  val microF1= 0.2585470085470085  val macroF1= 0.25457091131409465
 trained model:  META2 {'d1': 3, 'd2': 50, 'd3': 15, 'd4': 15, 'd5': 10}  epochs: 600  val loss= 0.07928942888975143  val accuracy= 0.2857142857142857  val microF1= 0.19871794871794876  val macroF1= 0.16602138253668963
 trained model:  META2 {'d1': 3, 'd2': 50, 'd3': 15, 'd4': 15, 'd5': 20}  epochs: 100  val loss= 0.05313444137573242  val accurac

AttributeError: 'Batch' object has no attribute 'u'

## Enzymes dataset benchmark

Paper  "Protein function prediction via graph kernels" , 91-95% accuracy on one-vs-rest classification of each class. No F1-scores

### References
- [4] K. M. Borgwardt, C. S. Ong, S. Schoenauer, S. V. N. Vishwanathan, A. J. Smola, and H. P. Kriegel. Protein function prediction via graph kernels. Bioinformatics, 21(Suppl 1):i47–i56, Jun 2005.
- [5] I. Schomburg, A. Chang, C. Ebeling, M. Gremse, C. Heldt, G. Huhn, and D. Schomburg. Brenda, the enzyme database: updates and major new developments. Nucleic Acids Research, 32D:431–433, 2004.

### Results
1. encapsulate all training, model selection,.. everything
2. present results with Pandas tables, and histograms
3. save models and results to disk, and load them later for testing
4. transform into a python module or package
5. prepare another notebook using the python module (prepare local and on collab)
6. test other GNN layers: GAT, GCN, GraphSAGE, Metalayer
7. do a good HP search
8. compare with published benchmarks
9. reload models and test again

### Pending:

- repeat for PPI, PROTEINS, IMDB and REDDIT
- look for published architectures?



## Resumin notebook

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import MessagePassing
#from torch_geometric.nn.conv.gated_graph_conv import GatedGraphConv
from torch_geometric.nn.glob.glob import global_mean_pool, global_add_pool
import torch.nn as nn
from torch.nn import Sequential as Seq, Linear as Lin, ReLU
from torch_scatter import scatter_mean
from torch_geometric.nn import MetaLayer
from torch_geometric.datasets import TUDataset, QM9, QM7b, PPI, Planetoid, KarateClub


from TFM_graph_classification import *

global device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# load dataset
thename='PROTEINS'
dataset = TUDataset(root='/tmp/'+thename, name=thename)
dataset = dataset.shuffle()
k = 3
n = len(dataset)
print(" n:",n," k folds=",k)
train_dataset, test_dataset = balancedDatasetSplit_slice(dataset, prop=0.8)
print("Datasets balancing: ")
printDatasetBalance(dataset )
printDatasetBalance(train_dataset )
printDatasetBalance(test_dataset )
print()


# load selection from saved models
modelsdict={}
modelsdict['models']=[]
modelsdict['best_models_list']=[]

filenames =[
    './models/enzymes/GGNN1__d1-25_d2-50_num_layers-2_aggr_type-mean_100_0.01_0.0005_32_date2019-03-31_05-34-35',
    './models/enzymes/GGNN1__d1-50_d2-50_num_layers-2_aggr_type-add_300_0.01_0.0005_32_date2019-03-31_05-34-35',
    './models/enzymes/GGNN1__d1-50_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_20-58-33',
    './models/enzymes/GGNN1__d1-50_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_21-14-13',
    './models/enzymes/GGNN1__d1-50_d2-50_num_layers-2_aggr_type-mean_600_0.01_0.0005_32_date2019-03-31_05-34-35',
    './models/enzymes/GGNN2__d1-100_d2-20_num_layers-2_aggr_type-mean_600_0.01_0.0005_32_date2019-03-31_08-41-00',
    './models/enzymes/GGNN2__d1-25_d2-20_num_layers-2_aggr_type-add_600_0.01_0.0005_32_date2019-03-31_08-41-00',
    './models/enzymes/GGNN2__d1-25_d2-50_num_layers-2_aggr_type-add_600_0.01_0.0005_32_date2019-03-31_08-41-00',
    './models/enzymes/GGNN3__d1-25_d2-20_num_layers-2_aggr_type-mean_600_0.01_0.0005_32_date2019-03-31_12-16-36',
    './models/enzymes/GGNN3__d1-25_d2-50_num_layers-2_aggr_type-add_300_0.01_0.0005_32_date2019-03-31_20-51-48',
    './models/enzymes/GGNN3__d1-25_d2-50_num_layers-2_aggr_type-add_300_0.01_0.0005_32_date2019-03-31_21-14-13',
    './models/enzymes/GGNN3__d1-25_d2-50_num_layers-2_aggr_type-add_300_0.01_0.0005_32_date2019-03-31_21-54-59',
    './models/enzymes/GGNN3__d1-25_d2-50_num_layers-2_aggr_type-add_300_0.01_0.0005_32_date2019-03-31_22-03-09',
    './models/enzymes/GGNN3__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_12-16-36',
    './models/enzymes/GGNN3__d1-50_d2-20_num_layers-2_aggr_type-mean_100_0.01_0.0005_32_date2019-03-31_12-16-36',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_100_0.01_0.0005_32_date2019-03-31_22-07-53',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_100_0.01_0.0005_32_date2019-03-31_22-09-13',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_100_0.01_0.0005_32_date2019-03-31_22-10-46',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_100_0.01_0.0005_32_date2019-03-31_22-11-38',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_100_0.01_0.0005_32_date2019-03-31_22-56-01',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_15-37-54',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_20-36-38',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_20-39-29',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_20-40-23',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_20-41-21',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_20-41-23',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_20-43-28',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_20-44-09',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_20-45-01',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_20-46-07',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_20-46-52',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_20-47-37',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_20-47-38',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_20-48-15',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_20-51-48',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_20-58-33',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_21-04-46',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_21-06-37',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_21-06-41',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_21-08-42',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_21-54-59',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-2_aggr_type-mean_600_0.01_0.0005_32_date2019-03-31_15-37-54',
    './models/enzymes/GGNN4__d1-25_d2-50_num_layers-4_aggr_type-mean_300_0.01_0.0005_32_date2019-03-31_15-37-54',
    './models/enzymes/META2__d1-3_d2-20_d3-15_d4-15_d5-10_300_0.01_0.0005_64_date2019-03-31_17-23-48',
    './models/enzymes/META2__d1-3_d2-20_d3-15_d4-15_d5-20_300_0.01_0.0005_64_date2019-03-31_17-23-48',
    './models/enzymes/META3__d1-3_d2-20_d3-15_d4-15_d5-10_200_0.01_0.0005_32_date2019-03-31_20-58-33',
    './models/enzymes/META3__d1-3_d2-20_d3-15_d4-15_d5-10_200_0.01_0.0005_32_date2019-03-31_21-14-13',
    './models/enzymes/META3__d1-3_d2-20_d3-15_d4-15_d5-10_200_0.01_0.0005_32_date2019-03-31_22-03-09',
    './models/enzymes/META3__d1-3_d2-20_d3-15_d4-15_d5-10_600_0.01_0.0005_64_date2019-03-31_17-05-20',
    './models/enzymes/META3__d1-3_d2-50_d3-15_d4-15_d5-20_600_0.01_0.0005_64_date2019-03-31_17-05-20',
]
for path in filenames:
    modeldict = loadModelFromFile(path)
    modelsdict['models'].append(modeldict)
    modelsdict['best_models_list'].append(modeldict)
    modelsdict['testing']={}
    modelsdict['best_models']={}

selection = [ (round(float(0),2), m) for m in modelsdict['best_models_list'] ]




In [ ]:
for i in range(len(selection)):    
    bmodel = final_model_train(selection[i][1], train_dataset)
    testresult = testModel(bmodel, test_dataset)
    modelsdict['testing'][bmodel.__class__.__name__+'_'+str(i)]=testresult

reportAllTest(modelsdict)
saveResults(modelsdict)